In [1]:
from pyspark.sql import SparkSession
from vars import *
from datetime import date
from functions import loadConfigs
import os
from pyspark.sql.functions import explode
from pyspark.sql.functions import lit
from pyspark.sql import functions as F
from delta import *

# minio credentials
minio_endpoint = "http://minio:9000"
minio_access_key = "iQekrU3nA33aKktu2GMo"
minio_secret_key = "1OGQ3m1lWEZVQ31xKfPCKmZVFVb2lXwCorjZr1d1"
minio_region = "us-east-1"
minio_bucket = "reddit-datalake"

builder = loadConfigs(SparkSession.builder)
spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [2]:
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [3]:
# today = date.today().strftime('%Y%m%d')
today = 20230610

input_folder = "gildings"
input_file = "gildings"
mode = "append"

df = spark.read.option("header", "true") \
    .parquet(f"s3a://{minio_bucket}/silver/{input_folder}/{input_file}_{today}")

In [4]:
df.show()

+-------+--------------------+------+------------------+-----------+---------+-------------+--------+
|post_id|              author|gilded|          gildings|gild_silver|gild_gold|gild_platinum|  dateid|
+-------+--------------------+------+------------------+-----------+---------+-------------+--------+
|145qm88|     Homar-The-Whale|     0|{null, null, null}|       null|     null|         null|20230610|
|145qdyb|     FionaWalliceFan|     0|{null, null, null}|       null|     null|         null|20230610|
|145na3a|       sunnymushroom|     0|{null, null, null}|       null|     null|         null|20230610|
|145simo|        shodaimezack|     0|{null, null, null}|       null|     null|         null|20230610|
|145l0wp|  povitryana_tryvoga|     0|{null, null, null}|       null|     null|         null|20230610|
|145f0u1|             AFoxGuy|     0|{null, null, null}|       null|     null|         null|20230610|
|145g3bh|            chris_ro|     0|{null, null, null}|       null|     null|    

In [5]:
df.printSchema()

root
 |-- post_id: string (nullable = true)
 |-- author: string (nullable = true)
 |-- gilded: long (nullable = true)
 |-- gildings: struct (nullable = true)
 |    |-- gid_1: long (nullable = true)
 |    |-- gid_2: long (nullable = true)
 |    |-- gid_3: long (nullable = true)
 |-- gild_silver: long (nullable = true)
 |-- gild_gold: long (nullable = true)
 |-- gild_platinum: long (nullable = true)
 |-- dateid: string (nullable = true)



In [6]:
author_df = df.groupBy("author").agg(
    F.sum("gild_silver").alias("total_gild_silver"),
    F.sum("gild_gold").alias("total_gild_gold"),
    F.sum("gild_platinum").alias("total_gild_platinum")
)

In [7]:
author_df.show()

+--------------------+-----------------+---------------+-------------------+
|              author|total_gild_silver|total_gild_gold|total_gild_platinum|
+--------------------+-----------------+---------------+-------------------+
|         sussywanker|                1|              2|                  2|
|     PoopyMcAllister|             null|           null|               null|
|      TrixoftheTrade|             null|           null|               null|
|        cooladriyana|             null|           null|               null|
|         lDeathWlshl|             null|           null|               null|
|              _alco_|                2|              5|                  1|
|Exciting_Potato_6717|             null|           null|               null|
|           jplodders|             null|              1|                  1|
|          qingfengdi|             null|           null|               null|
|         BigHoneyBot|             null|           null|               null|

In [8]:
author_df = author_df.withColumn(
    'total_gildings',
    F.col('total_gild_silver') + F.col('total_gild_gold') + F.col('total_gild_platinum')
).orderBy('total_gildings',ascending=False)

In [9]:
author_df.show()

+--------------------+-----------------+---------------+-------------------+--------------+
|              author|total_gild_silver|total_gild_gold|total_gild_platinum|total_gildings|
+--------------------+-----------------+---------------+-------------------+--------------+
|            TitusRex|                1|             18|                  8|            27|
|              _alco_|                2|              5|                  1|             8|
|          OBLIVIATER|                1|              5|                  2|             8|
|         sussywanker|                1|              2|                  2|             5|
|     PoopyMcAllister|             null|           null|               null|          null|
|      TrixoftheTrade|             null|           null|               null|          null|
|        cooladriyana|             null|           null|               null|          null|
|         lDeathWlshl|             null|           null|               null|    

In [10]:
post_df = df.groupBy("post_id").agg(
    F.sum("gild_silver").alias("total_gild_silver"),
    F.sum("gild_gold").alias("total_gild_gold"),
    F.sum("gild_platinum").alias("total_gild_platinum")
)

In [11]:
post_df = post_df.withColumn(
    'total_gildings',
    F.col('total_gild_silver') + F.col('total_gild_gold') + F.col('total_gild_platinum')
)

In [12]:
post_df.orderBy('total_gildings',ascending=False).show()

+-------+-----------------+---------------+-------------------+--------------+
|post_id|total_gild_silver|total_gild_gold|total_gild_platinum|total_gildings|
+-------+-----------------+---------------+-------------------+--------------+
|145efj4|                1|             18|                  8|            27|
|145ea4f|                2|              5|                  1|             8|
|145vns0|                1|              5|                  2|             8|
|145hyey|                1|              2|                  2|             5|
|145ji4h|             null|           null|               null|          null|
|145t1sa|             null|           null|               null|          null|
|145nhgv|             null|           null|               null|          null|
|145opim|             null|           null|               null|          null|
|145p0w0|             null|           null|               null|          null|
|145bv3e|             null|           null|         

In [13]:
from pyspark.sql.functions import current_date, date_format

author_df = author_df.withColumn("updated_at", date_format(current_date(), "yyyyMMdd"))

In [14]:
author_df.show()

+--------------------+-----------------+---------------+-------------------+--------------+----------+
|              author|total_gild_silver|total_gild_gold|total_gild_platinum|total_gildings|updated_at|
+--------------------+-----------------+---------------+-------------------+--------------+----------+
|            TitusRex|                1|             18|                  8|            27|  20230611|
|              _alco_|                2|              5|                  1|             8|  20230611|
|          OBLIVIATER|                1|              5|                  2|             8|  20230611|
|         sussywanker|                1|              2|                  2|             5|  20230611|
|     PoopyMcAllister|             null|           null|               null|          null|  20230611|
|      TrixoftheTrade|             null|           null|               null|          null|  20230611|
|        cooladriyana|             null|           null|               nu

In [15]:
author_df.columns

['author',
 'total_gild_silver',
 'total_gild_gold',
 'total_gild_platinum',
 'total_gildings',
 'updated_at']

In [16]:
output_folder = "author_gildings"
output_file = "author_gildings"

In [18]:
author_df.write.format("delta").mode("overwrite").save(f"s3a://{minio_bucket}/gold/{output_folder}/{output_file}_{today}")